# Replace missing ages
#### We analyze the passenger survival data from the sinking of the Titanic to discover how survival, sex, age, and passenger class are correlated. Was it "women and children first" into the lifeboats? The CVS file `TitanicSurvivors.csv` contains one record per passenger, and each passenger has a name (maximum 31 characters) and three attributes: survived (yes or no), sex (male or female), age, and passenger class (1st, 2nd, or 3rd). The data is sorted first by passenger class and then by name. The first few lines of the file:
```
"","survived","sex","age","passengerClass"
"Allen, Miss. Elisabeth Walton","yes","female",29,"1st"
"Allison, Master. Hudson Trevor","yes","male",0.916700006,"1st"
"Allison, Miss. Helen Loraine","no","female",2,"1st"
"Allison, Mr. Hudson Joshua Crei","no","male",30,"1st"
"Allison, Mrs. Hudson J C (Bessi","no","female",25,"1st"
"Anderson, Mr. Harry","yes","male",48,"1st"
"Andrews, Miss. Kornelia Theodos","yes","female",63,"1st"
"Andrews, Mr. Thomas Jr","no","male",39,"1st"
"Appleton, Mrs. Edward Dale (Cha","yes","female",53,"1st"
"Artagaveytia, Mr. Ramon","no","male",71,"1st"
"Astor, Col. John Jacob","no","male",47,"1st"
"Astor, Mrs. John Jacob (Madelei","yes","female",18,"1st"
"Aubart, Mme. Leontine Pauline","yes","female",24,"1st"
"Barber, Miss. Ellen Nellie","yes","female",26,"1st"
"Barkworth, Mr. Algernon Henry W","yes","male",80,"1st"
"Baumann, Mr. John D","no","male",NA,"1st"
"Baxter, Mr. Quigg Edmond","no","male",24,"1st"
"Baxter, Mrs. James (Helene DeLa","yes","female",50,"1st"
```
#### The name column does not have a header, but the passenger names are irrelevant to the analysis. Babies have fractional ages (for example, the first Allison). Not all ages were recorded, and missing ages were entered as `NA` (for example, Baumann).
#### Because our analysis includes the passengers' ages, what should we do about the missing ages? We can either
- #### Throw out the records with missing ages. But that can cause problems from losing other information about those passengers.
- #### Replace each missing ages with a placeholder value. But doing so can introduce bias into the analysis.

# We decide to replace each missing age with an ***average age***. 
#### What is the best average to use?
#### We should replace each missing age by an average age that is ***most representative*** of the known ages in its subgroup -- the average age of passengers with similar attribute values. However, we should not replace a number of missing ages by an average of fewer known ages. In other words, out of 10 ages, it should be OK to replace 2 missing ages by the average of the 8 known ages, but not replace 8 missing ages by the average of the 2 known ages.
#### We must ***explore the data*** to find the best average to use.

In [1]:
import csv
import numpy as np
import pandas as pd
from pandas import DataFrame

## Global constants

In [2]:
CSV_FILE = 'TitanicSurvival.csv'

CLASS_COUNT = 4
CLASS_1     = 0
CLASS_2     = 1
CLASS_3     = 2

SEX_COUNT  = 2
SEX_FEMALE = 0
SEX_MALE   = 1

SURVIVED_COUNT = 2
SURVIVED_NO    = 0
SURVIVED_YES   = 1

AGE_COUNT = 2
AGE_NA    = 0
AGE_OK    = 1

## Read the Titanic Survival CSV data file
#### To aid the analysis, transform each record field:
- survived
    - 0 = no
    - 1 = yes
- sex
    - 0 = female
    - 1 = male
- age
    - -1 = NA
- passenger class
    - 1 = 1st
    - 2 = 2nd
    - 3 = 3rd
#### The lists `Survived`, `Sex`, `Age`, and `Pclass` will contain values from all the records.

In [3]:
def read_data(filename):
    """
    Read the Titanic survival data. Only use rows
    where age is not NA. Transform sex to 0 for female
    and 1 for male. Transform passenger class to 1, 2, or 3.
    Transform survived to 0 for no and 1 for yes.
    Return lists Pclass, Sex, Age, and Survived.
    """
    first = True
    i = 0

    Name      = []
    Pclass    = []
    Sex       = []
    Survived  = []
    Age       = []

    with open(filename, newline='') as csv_file:
        data = csv.reader(csv_file, delimiter=',', quotechar='"')

        for row in data:
            
            # Skip the first (header) row.
            if not first:
                name, survived, sex, age, pclass = row
                
                age      = -1 if age== 'NA' else float(age)

                survived = 0 if survived == 'no'     else \
                           1
                sex      = 0 if sex      == 'female' else \
                           1
                pclass   = 1 if pclass   == '1st'    else \
                           2 if pclass   == '2nd'    else \
                           3

                Age.append(age)
                Sex.append(sex)
                Survived.append(survived)
                Pclass.append(pclass)

            first = False
            
    return Survived, Sex, Age, Pclass

In [4]:
Survived, Sex, Age, Pclass = read_data(CSV_FILE)

## Count ages
#### Count how many ages are missing or not. If we consider survived, sex, age (missing or not), and passenger class attributes to be dimensions, we can represent the data as a 4-dimensional ***hypercube*** `age_counts` and use the transformed field values to index into it. The hypercube will allow us to easily sum along each attribute.

In [5]:
def count_ages(Pclass, Sex, Survived, Age):
    """
    Create a 4-dimensional hypercube to count ages in
    each of the class, sex, survived, and age dimensions.
    Return the hypercube.
    """
    multidimensional_list = \
        [0]*CLASS_COUNT*SEX_COUNT*SURVIVED_COUNT*AGE_COUNT

    hypercube = np.array(multidimensional_list)\
        .reshape(CLASS_COUNT, SEX_COUNT, SURVIVED_COUNT, AGE_COUNT)
    
    for klass, sex, survived, age in zip(Pclass, Sex, Survived, Age):
        if age < 0:
            hypercube[klass][sex][survived][AGE_NA] += 1
        else:
            hypercube[klass][sex][survived][AGE_OK] += 1
            
    return hypercube

In [6]:
age_counts = count_ages(Pclass, Sex, Survived, Age)
print('\nHYPERCUBE OF AGE COUNTS')
print(age_counts)


HYPERCUBE OF AGE COUNTS
[[[[  0   0]
   [  0   0]]

  [[  0   0]
   [  0   0]]]


 [[[  0   5]
   [ 11 128]]

  [[ 20  98]
   [  8  53]]]


 [[[  1  11]
   [  2  92]]

  [[ 11 135]
   [  2  23]]]


 [[[ 30  80]
   [ 34  72]]

  [[128 290]
   [ 16  59]]]]


## Age counts by passenger class
#### In a dataframe, show the counts of missing ages and their percentages by passenger class.

In [7]:
def print_age_counts_by_class(age_counts):
    """
    Display the counts and percentages of missing ages
    in each class.
    """
    table = []
    
    # Overall
    all_count    = np.sum(age_counts[:, :, :, :])
    all_na_count = np.sum(age_counts[:, :, :, AGE_NA])
    all_na_pct   = 100 * all_na_count/all_count
    
    table.append(['all', all_count, all_na_count, all_na_pct])
    
    # By passenger class
    for klass in range(1, CLASS_COUNT):
        class_count    = np.sum(age_counts[klass, :, :, :])
        class_na_count = np.sum(age_counts[klass, :, :, AGE_NA])
        class_na_pct   = 100 * class_na_count/class_count
        
        table.append([klass, class_count, class_na_count, class_na_pct])
        
    df = DataFrame(table)
    df.columns = ['class', 'count', 'NAs', '% NAs']
    display(df)

In [8]:
print('\n% MISSING AGES BY CLASS')
print_age_counts_by_class(age_counts)


% MISSING AGES BY CLASS


,class,count,NAs,% NAs
0,all,1309,263,20.091673
1,1,323,39,12.074303
2,2,277,16,5.776173
3,3,709,208,29.337094


## Age counts by sex by passenger class
#### In a dataframe, show the counts of missing ages and their percentages by sex within each passenger class.

In [9]:
def print_age_counts_by_sex(age_counts):
    """
    Display the counts and percentages of missing ages
    in the subgroups by sex by class.
    """
    table = []
    
    # By passenger class
    for klass in range(1, CLASS_COUNT):
        
        # Female NA
        female_count    = np.sum(age_counts[klass, SEX_FEMALE, :, :])
        female_na_count = np.sum(age_counts[klass, SEX_FEMALE, :, AGE_NA])
        female_na_pct   = 100 * female_na_count/female_count
        
        # Male NA
        male_count    = np.sum(age_counts[klass, SEX_MALE, :, :])
        male_na_count = np.sum(age_counts[klass, SEX_MALE, :, AGE_NA])
        male_na_pct   = 100 * male_na_count/male_count

        table.append([klass, female_count, female_na_count, female_na_pct,
                             male_count,   male_na_count,   male_na_pct])

    df = DataFrame(table)
    df.columns = ['class', 'female count', 'female NAs', 'female % NAs',
                           'male count',   'male NAs',   'male % NAs']
    display(df)

In [10]:
print('\n% MISSING AGES BY SEX BY CLASS')
print_age_counts_by_sex(age_counts)


% MISSING AGES BY SEX BY CLASS


,class,female count,female NAs,female % NAs,male count,male NAs,male % NAs
0,1,144,11,7.638889,179,28,15.642458
1,2,106,3,2.830189,171,13,7.602339
2,3,216,64,29.629630,493,144,29.208925


## Age counts by survival by sex by passenger class
#### In a dataframe, show the counts of missing ages and their percentages by survival by sex within each passenger class.

In [11]:
def print_age_counts_by_survived(age_counts):
    """
    Display the counts and percentages of missing ages
    in the subgroups by survived by sex by class.
    """
    table = []
    
    # By passenger class
    for klass in range(1, CLASS_COUNT):
        
        #Female perished NA
        female_perished_count    = np.sum(age_counts[klass, SEX_FEMALE, SURVIVED_NO, :])
        female_perished_na_count = np.sum(age_counts[klass, SEX_FEMALE, SURVIVED_NO, AGE_NA])
        female_perished_na_pct   = 100 * female_perished_na_count/female_perished_count
        
        # Male perished NA
        male_perished_count    = np.sum(age_counts[klass, SEX_MALE, SURVIVED_NO, :])
        male_perished_na_count = np.sum(age_counts[klass, SEX_MALE, SURVIVED_NO, AGE_NA])
        male_perished_na_pct   = 100 * male_perished_na_count/male_perished_count
        
        table.append([klass, 'no', female_perished_count, 
                                   female_perished_na_count, 
                                   female_perished_na_pct,
                                   male_perished_count, 
                                   male_perished_na_count, 
                                   male_perished_na_pct])
        
        # Female survived NA
        female_survived_count    = np.sum(age_counts[klass, SEX_FEMALE, SURVIVED_YES, :])
        female_survived_na_count = np.sum(age_counts[klass, SEX_FEMALE, SURVIVED_YES, AGE_NA])
        female_survived_na_pct   = 100 * female_survived_na_count/female_survived_count
        
        # Male survived NA
        male_survived_count    = np.sum(age_counts[klass, SEX_MALE, SURVIVED_YES, :])
        male_survived_na_count = np.sum(age_counts[klass, SEX_MALE, SURVIVED_YES, AGE_NA])
        male_survived_na_pct   = 100 * male_survived_na_count/male_survived_count
        
        table.append([klass, 'yes', female_survived_count, 
                                    female_survived_na_count, 
                                    female_survived_na_pct,
                                    male_survived_count, 
                                    male_survived_na_count, 
                                    male_survived_na_pct])
        
    df = DataFrame(table)
    df.columns = ['class', 'survived', 'female count', 'female NAs', 'female % NAs',
                                       'male count',   'male NAs',   'male % NAs']
    display(df)

In [12]:
print('\n% MISSING AGES BY SURVIVAL BY SEX BY CLASS')
print_age_counts_by_survived(age_counts)


% MISSING AGES BY SURVIVAL BY SEX BY CLASS


,class,survived,female count,female NAs,female % NAs,male count,male NAs,male % NAs
0,1,no,5,0,0.000000,118,20,16.949153
1,1,yes,139,11,7.913669,61,8,13.114754
2,2,no,12,1,8.333333,146,11,7.534247
3,2,yes,94,2,2.127660,25,2,8.000000
4,3,no,110,30,27.272727,418,128,30.622010
5,3,yes,106,34,32.075472,75,16,21.333333


### The above tables show that it should be OK to replace each missing age by the average of known ages in its smallest subgroup. The percentage of missing ages in each subgroup should be significantly less than 50%. We see that the replacements in 3rd class (> 30% and > 21%) will be more probematical than in the other classes.

## Skip rows where age is `NA`
#### Compute some baseline statistics with the original data but without the missing ages.

In [13]:
def skip_na(Survived, Sex, Age, Pclass):
    """
    Return copies of the Survived, Sex, Age, and Pclass lists
    without the rows where age is NA.
    """
    Survived_no_na = []
    Sex_no_na      = []
    Age_no_na      = []
    Pclass_no_na   = []
    
    for survived, sex, age, pclass in zip(Survived, Sex, Age, Pclass):
        if age >= 0:
            Survived_no_na.append(survived)
            Sex_no_na.append(sex)
            Age_no_na.append(age)
            Pclass_no_na.append(pclass)
            
    return Survived_no_na, Sex_no_na, Age_no_na, Pclass_no_na

In [14]:
Survived_no_na, Sex_no_na, Age_no_na, Pclass_no_na = \
                            skip_na(Survived, Sex, Age, Pclass)

### Cache of ages
#### Dictionary `averages` will cache age counts in each dimension.

In [15]:
averages = {}

## Average ages by passenger class
#### Calculate the average age of each class.

In [16]:
def average_age_class(target_class, classes, ages, averages):
    """
    Calculate the average age of the target passenger class 
    using the classes and ages lists and return the average.
    """
    if target_class == 0:
        avg = sum(ages)/len(ages)  # overall
    else:
        count = 0
        total = 0

        # By target passenger class
        for klass, age in zip(classes, ages):
            if klass == target_class:    
                count += 1
                total += age

        avg = total/count
        
    return avg

#### Print the average age of each class.

In [17]:
def print_average_age_class(classes, ages, averages):
    """
    Print the average age of each passenger class using
    the classes and ages lists and the averages dictionary.
    """
    row = []
    
    for klass in range(CLASS_COUNT):
        row.append(average_age_class(klass, classes, ages, 
                                     averages))

    df = DataFrame([row])
    df.columns = ['all', '1st', '2nd', '3rd']
    display(df)

### Our baseline is the average ages by class without the missing ages.

In [18]:
print('\nAVERAGE AGE BY CLASS'
      '\nWITHOUT MISSING AGES')
print_average_age_class(Pclass_no_na, Age_no_na, averages)


AVERAGE AGE BY CLASS
WITHOUT MISSING AGES


,all,1st,2nd,3rd
0,29.881135,39.159918,29.506705,24.816367


#### Replace the missing age in each class.

In [19]:
def replace_by_class(classes, ages, averages):
    """
    Replace each missing age by the passenger class average using
    the passenger classes and ages lists and the averages dictionary.
    """
    clean = []
    
    for klass, age in zip(classes, ages):
        clean.append(age if age >= 0 \
                     else average_age_class(klass, classes, 
                                            ages, averages))

    return clean

In [20]:
print('\nAVERAGE AGE BY CLASS'
      '\nAFTER REPLACEMENTS WITH CLASS AVERAGES')
Age_clean_class = replace_by_class(Pclass, Age, averages)
print_average_age_class(Pclass, Age_clean_class, averages)


AVERAGE AGE BY CLASS
AFTER REPLACEMENTS WITH CLASS AVERAGES


,all,1st,2nd,3rd
0,27.978735,38.574431,29.404922,22.594443


## Average age of each sex subgroup
#### Calculate the average age of each sex subgroup in each class.

In [21]:
def average_age_sex(target_class, target_sex, 
                    classes, sexes, ages, averages):
    """
    Retrieve the average age of the target sex within 
    the target passenger class from the averages dictionary 
    if it's in there. Otherwise, calculate the average using 
    the classes, sexes, and ages lists and enter it into 
    the dictionary. Return the average.
    """
    key = f'{target_class}-{target_sex}'
    
    if key in averages:
        return averages[key]
    
    if target_class == 0:
        avg = sum(ages)/len(ages)  # overall
    else:
        count = 0
        total = 0
    
        # By target passenger class and sex
        for klass, sex, age in zip(classes, sexes, ages):
            if     (klass == target_class) \
               and (sex   == target_sex):
                count += 1
                total += age

        avg = total/count
        
    averages[key] = avg
    return avg

#### Print the average age of each sex subgroup.

In [22]:
def print_average_age_sex(classes, sexes, ages, averages):
    """
    Print the average age of each sex in each passenger 
    class using the classes, sexes, and ages lists and  
    the averages dictionary.
    """
    table = []
    
    for klass in range(CLASS_COUNT):
        table.append(['all' if klass == 0 else klass,
                      average_age_sex(klass, SEX_FEMALE, 
                                      classes, sexes, 
                                      ages, averages),
                      average_age_sex(klass, SEX_MALE, 
                                      classes, sexes, 
                                      ages, averages)])

    df = DataFrame(table)
    df.columns=['class', 'female average', 'male average']
    display(df)

In [23]:
print('\nAVERAGE AGES BY SEX BY CLASS'
      '\nWITHOUT MISSING AGES')
print_average_age_sex(Pclass_no_na, Sex_no_na, Age_no_na, averages)


AVERAGE AGES BY SEX BY CLASS
WITHOUT MISSING AGES


,class,female average,male average
0,all,29.881135,29.881135
1,1,37.037594,41.029250
2,2,27.499191,30.815401
3,3,22.185307,25.962273


#### Replace the missing age in each sex subgroup.

In [24]:
def replace_by_sex(classes, sexes, ages, averages):
    """
    Replace each missing age by the average age in each sex 
    subgroup within each class using the classes, sexes, and 
    ages lists and the averages dictionary.
    """
    clean = []
    
    for klass, sex, age in zip(classes, sexes, ages):
        clean.append(age if age >= 0 \
                     else average_age_sex(klass, sex, 
                                          classes, sexes, 
                                          ages, averages))

    return clean

In [25]:
print('\nAVERAGE AGES BY SEX BY CLASS'
      '\nAFTER REPLACEMENTS WITH SEX SUBGROUP AVERAGES')
Age_clean_sex = replace_by_sex(Pclass, Sex, Age, averages)
print_average_age_class(Pclass, Age_clean_sex, averages)


AVERAGE AGES BY SEX BY CLASS
AFTER REPLACEMENTS WITH SEX SUBGROUP AVERAGES


,all,1st,2nd,3rd
0,29.376183,39.249688,29.546382,24.811604


## Average age of each survived subgroup
#### Calculate the average age of each survived subgroup in each sex subgroup in each passenger class. by survival status by sex by passenger class.

In [26]:
def average_age_survived(target_class, target_sex, target_survived,
                         classes, sexes, surviveds, ages, averages):
    """
    Retrieve the average age of the target survived subgroup
    in the target sex subgroup in the target passenger class 
    from the averages dictionary if it's in there.  Otherwise, 
    calculate the average using the classes, sexes, surviveds, 
    and ages lists and enter it into the dictionary. Return 
    the average.
    """
    key = f'{target_class}-{target_sex}-{target_survived}'
    
    if key in averages:
        return averages[key]
    
    if target_class == 0:
        avg = sum(ages)/len(ages)  # overall
    else:
        count = 0
        total = 0
    
        # By target passenger class, sex, and survival
        for klass, sex, survived, age \
                       in zip(classes, sexes, surviveds, ages):
            if     (klass    == target_class) \
               and (sex      == target_sex) \
               and (survived == target_survived):
                count += 1
                total += age

        avg = total/count
        
    averages[key] = avg
    return avg

#### Print the average age of each survived subgroup.

In [27]:
def print_average_age_survived(classes, sexes, surviveds, ages, averages):
    """
    Print the average age of each survived subgroup in each 
    sex subgroup in each passenger class using the classes, 
    sexes, surviveds, and ages lists and the averages dictionary.
    """
    table = []

    for klass in range(1, CLASS_COUNT):
        table.append([klass,
                      average_age_survived(klass, SEX_FEMALE, SURVIVED_NO,
                                           classes, sexes, surviveds, ages, 
                                           averages),
                      average_age_survived(klass, SEX_MALE, SURVIVED_NO,
                                           classes, sexes, surviveds, ages, 
                                           averages),
                      average_age_survived(klass, SEX_FEMALE, SURVIVED_YES,
                                           classes, sexes, surviveds, ages, 
                                           averages),
                      average_age_survived(klass, SEX_MALE, SURVIVED_YES,
                                           classes, sexes, surviveds, ages, 
                                           averages)])

    df = DataFrame(table)
    df.columns=['class', 'female perished', 'male perished', 
                         'female survived', 'male survived']
    display(df)

In [28]:
print('\nAVERAGE AGES BY SURVIVED BY SEX BY CLASS'
      '\nWITHOUT MISSING AGES')    
print_average_age_survived(Pclass_no_na, Sex_no_na, 
                           Survived_no_na, Age_no_na, averages)


AVERAGE AGES BY SURVIVED BY SEX BY CLASS
WITHOUT MISSING AGES


,class,female perished,male perished,female survived,male survived
0,1,35.200000,43.658163,37.109375,36.168240
1,2,34.090909,33.092593,26.711051,17.449274
2,3,23.418750,26.679598,20.814815,22.436441


#### Replace the missing ages in each survived subgroup.

In [29]:
def replace_by_survived(classes, sexes, surviveds, ages, averages):
    """
    Replace each missing age by the average of the survived subggroup
    in each survival subgroup in each sex within each class using 
    the classes, sexes, and ages lists and the averages dictionary.
    """
    clean = []
    
    for klass, sex, survived, age \
    in zip(classes, sexes, surviveds, ages):
        clean.append(age if age >= 0 \
                     else average_age_survived(klass, sex, 
                                               survived, classes, 
                                               sexes, surviveds,
                                               ages, averages))

    return clean

In [30]:
print('\nAVERAGE AGES BY SURVIVED BY SEX BY CLASS'
      '\nAFTER REPLACEMENTS WITH SURVIVED SUBGROUP AVERAGES')
Age_clean_survived = replace_by_survived(Pclass, Sex, Survived, Age, averages)
print_average_age_class(Pclass, Age_clean_survived, averages)


AVERAGE AGES BY SURVIVED BY SEX BY CLASS
AFTER REPLACEMENTS WITH SURVIVED SUBGROUP AVERAGES


,all,1st,2nd,3rd
0,29.409509,39.294517,29.558412,24.848008


## Correlation matrix

In [31]:
def print_correlation_matrix(classes, sexes, surviveds, ages):
    """
    Display the pairwise correlation matrix.
    """
    M = np.array([classes, sexes, surviveds, ages]).T

    df = DataFrame(M)
    df.columns = ['Class', 'Sex', 'Survived', 'Age']

    display(df.corr(method='pearson', min_periods=1))

In [32]:
print('\nPAIRWISE CORRELATION MATRIX WITHOUT MISSING AGES')
print_correlation_matrix(Pclass_no_na, Sex_no_na, 
                         Survived_no_na, Age_no_na)

print('\nPAIRWISE CORRELATION MATRIX WITH REPLACED AGES')
print_correlation_matrix(Pclass, Sex, Survived, Age_clean_survived)


PAIRWISE CORRELATION MATRIX WITHOUT MISSING AGES


,Class,Sex,Survived,Age
Class,1.000000,0.144695,-0.320486,-0.408106
Sex,0.144695,1.000000,-0.538000,0.063646
Survived,-0.320486,-0.538000,1.000000,-0.055513
Age,-0.408106,0.063646,-0.055513,1.000000



PAIRWISE CORRELATION MATRIX WITH REPLACED AGES


,Class,Sex,Survived,Age
Class,1.000000,0.124617,-0.312469,-0.444002
Sex,0.124617,1.000000,-0.528693,0.080752
Survived,-0.312469,-0.528693,1.000000,-0.060032
Age,-0.444002,0.080752,-0.060032,1.000000


## Main

In [33]:
def main():
    Survived, Sex, Age, Pclass = read_data(CSV_FILE)
    
    Survived_no_na, Sex_no_na, Age_no_na, Pclass_no_na = \
                                skip_na(Survived, Sex, Age, Pclass)
    
    age_counts = count_ages(Pclass, Sex, Survived, Age)
    print('\nHYPERCUBE OF AGE COUNTS')
    print(age_counts)

    print('\n% MISSING AGES BY CLASS')
    print_age_counts_by_class(age_counts)
    
    print('\n% MISSING AGES BY SEX BY CLASS')
    print_age_counts_by_sex(age_counts)
    
    print('\n% MISSING AGES BY SURVIVAL BY SEX BY CLASS')
    print_age_counts_by_survived(age_counts)
    
    averages = {}
    
    print('\nAVERAGE AGE BY CLASS'
          '\nWITHOUT MISSING AGES')
    print_average_age_class(Pclass_no_na, Age_no_na, averages)
    
    print('\nAVERAGE AGE BY CLASS'
          '\nAFTER REPLACEMENTS WITH CLASS AVERAGES')
    Age_clean_class = replace_by_class(Pclass, Age, averages)
    print_average_age_class(Pclass, Age_clean_class, averages)
    
    print('\nAVERAGE AGES BY SEX BY CLASS'
          '\nWITHOUT MISSING AGES')
    print_average_age_sex(Pclass_no_na, Sex_no_na, Age_no_na, averages)
    
    print('\nAVERAGE AGES BY SEX BY CLASS'
          '\nAFTER REPLACEMENTS WITH SEX SUBGROUP AVERAGES')
    Age_clean_sex = replace_by_sex(Pclass, Sex, Age, averages)
    print_average_age_class(Pclass, Age_clean_sex, averages)
    
    print('\nAVERAGE AGES BY SURVIVED BY SEX BY CLASS'
          '\nWITHOUT MISSING AGES')    
    print_average_age_survived(Pclass_no_na, Sex_no_na, 
                               Survived_no_na, Age_no_na, averages)

    print('\nAVERAGE AGES BY SURVIVED BY SEX BY CLASS'
          '\nAFTER REPLACEMENTS WITH SURVIVED SUBGROUP AVERAGES')
    Age_clean_survived = replace_by_survived(Pclass, Sex, Survived, 
                                             Age, averages)
    print_average_age_class(Pclass, Age_clean_survived, averages)
    
    print('\nPAIRWISE CORRELATION MATRIX WITHOUT MISSING AGES')
    print_correlation_matrix(Pclass_no_na, Sex_no_na, 
                             Survived_no_na, Age_no_na)
    
    print('\nPAIRWISE CORRELATION MATRIX WITH REPLACED AGES')
    print_correlation_matrix(Pclass, Sex, Survived, Age_clean_survived)

In [34]:
main()


HYPERCUBE OF AGE COUNTS
[[[[  0   0]
   [  0   0]]

  [[  0   0]
   [  0   0]]]


 [[[  0   5]
   [ 11 128]]

  [[ 20  98]
   [  8  53]]]


 [[[  1  11]
   [  2  92]]

  [[ 11 135]
   [  2  23]]]


 [[[ 30  80]
   [ 34  72]]

  [[128 290]
   [ 16  59]]]]

% MISSING AGES BY CLASS


,class,count,NAs,% NAs
0,all,1309,263,20.091673
1,1,323,39,12.074303
2,2,277,16,5.776173
3,3,709,208,29.337094



% MISSING AGES BY SEX BY CLASS


,class,female count,female NAs,female % NAs,male count,male NAs,male % NAs
0,1,144,11,7.638889,179,28,15.642458
1,2,106,3,2.830189,171,13,7.602339
2,3,216,64,29.629630,493,144,29.208925



% MISSING AGES BY SURVIVAL BY SEX BY CLASS


...

,all,1st,2nd,3rd
0,29.409509,39.294517,29.558412,24.848008



PAIRWISE CORRELATION MATRIX WITHOUT MISSING AGES


,Class,Sex,Survived,Age
Class,1.000000,0.144695,-0.320486,-0.408106
Sex,0.144695,1.000000,-0.538000,0.063646
Survived,-0.320486,-0.538000,1.000000,-0.055513
Age,-0.408106,0.063646,-0.055513,1.000000



PAIRWISE CORRELATION MATRIX WITH REPLACED AGES


,Class,Sex,Survived,Age
Class,1.000000,0.124617,-0.312469,-0.444002
Sex,0.124617,1.000000,-0.528693,0.080752
Survived,-0.312469,-0.528693,1.000000,-0.060032
Age,-0.444002,0.080752,-0.060032,1.000000


## Conclusion
#### As shown by the two pairwise correlation matrices, the age attribute correlates **slightly better** with each of the other attributes if we replace each missing person's age by the survived subgroup's average age. Age is the worse predictor of survival. Age is much better at predicting a person's passenger class. A person's sex is the best predictor of survival.

#### (c) 2023 by Ronald Mak